In [1]:
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import csv
from torchvision import models, transforms

In [2]:
class MyDigitDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        image_paths = []
        for ext in ('*.jpg', '*.jpeg', '*.png'):
            image_paths.extend(glob.glob(os.path.join(root_dir, '*', 'hand_written_digit', '*', ext)))
        self.image_paths = image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = int(os.path.basename(img_path).split('_')[0])
        if self.transform:
            image = self.transform(image)
        return image, label
class MyTestDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.transform = transform
        self.image_paths = []
        for ext in ('*.jpg', '*.jpeg', '*.png'):
            self.image_paths.extend(glob.glob(os.path.join(folder, ext)))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        while idx < len(self.image_paths):
            img_path = self.image_paths[idx]
            try:
                image = Image.open(img_path).convert("RGB")
                if self.transform:
                    image = self.transform(image)
                return image, img_path
            except Exception:
                idx += 1  # Bỏ qua ảnh lỗi, thử ảnh tiếp theo
        raise IndexError("No valid image found at or after index {}".format(idx))

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),                   # Resize đúng input MobileNet
    transforms.ToTensor(),                           
])
# MobileNet yêu cầu input 224x224 pixels
# ToTensor() tự động chia pixel values cho 255 (chuẩn hóa về [0,1])

In [ ]:
root_path = r'D:\CS114_P21\Doan\train'
traindata = MyDigitDataset(root_path, transform=transform)
dataloader = DataLoader(traindata, batch_size=64, shuffle=True, num_workers=0)

test_folder = r'D:\CS114_P21\Doan\test'
testdata = MyTestDataset(test_folder, transform=transform)
test_loader = DataLoader(testdata, batch_size=64,num_workers=0, shuffle=False)
# Tạo dataset objects từ các class đã định nghĩa
# DataLoader chia data thành batches, shuffle training data
# num_workers=0: không dùng multiprocessing (tránh lỗi trên Windows)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(pretrained=False) # true = trong số đã train, false = tự train
model.classifier[1] = nn.Linear(model.last_channel, 10)  # 10 chữ số (0–9)
model.to(device)
# pretrained=False: Train từ đầu, không dùng ImageNet weights
# Thay classifier cuối cùng từ 1000 classes (ImageNet) thành 10 classes (digits 0-9)

In [ ]:
#training loop
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):  # tăng nếu muốn
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/10] - Loss: {running_loss:.4f} - Acc: {100 * correct / total:.2f}%")
#     Forward pass: Ảnh đi qua model → outputs (logits)
# Loss calculation: So sánh outputs với ground truth labels
# Backward pass: Tính gradients của loss theo weights
# Weight update: Optimizer cập nhật weights theo gradients
# Metrics: Tính accuracy cho epoch


In [ ]:
model.eval()
results = []

with torch.no_grad():
    for images, img_paths in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        preds = preds.cpu().numpy()  # chuyển về numpy để dễ xử lý

        for path, pred in zip(img_paths, preds):
            filename = os.path.basename(path)
            results.append([filename, int(pred)])

# Ghi ra file CSV
with open('MOBILENETWECODE.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['filename', 'prediction'])  # header
    writer.writerows(results)
# Chuyển model sang evaluation mode
# Tắt gradient computation
# Chạy forward pass trên test data
# Lấy prediction (class có score cao nhất)
# Lưu kết quả (filename, prediction) vào CSV